In [1]:
import streamlit as st
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
from pymongo import MongoClient
import urllib.parse
import requests
from bertopic import BERTopic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import deepcut
import unicodedata
from pythainlp.util import normalize
import torch
import csv

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Asus\AppData\Local\Programs\Python\Python311\

In [2]:
#import data
sbert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

data = pd.read_csv('articles_data.csv')

with open('corpus_embeddings.pickle', 'rb') as file:
    corpus_embeddings = pickle.load(file)

In [3]:
queries = 'ไอเป็นเลือด'

index_lst = []
score_lst = []

query_embedding = sbert_model.encode(queries, convert_to_tensor=True)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)
hits = hits[0]
for hit in hits:
    index_lst.append(hit['corpus_id'])
    score_lst.append(hit['score'])

sbert_searched = data.iloc[index_lst]
sbert_searched['score'] = score_lst



In [5]:
sbert_searched[['url','title','content','category','owner source','score']]

,url,title,content,category,owner source,score
909,https://www.petcharavejhospital.com/th/Article...,ไอเป็นเลือด สัญญาณของโรคร้ายที่ต้องตรวจหา,ไอเป็นเลือด (Haemoptysis) คือ อวัยวะทางเดินหาย...,health-articles,petcharavej hospital,0.682584
902,https://www.petcharavejhospital.com/th/Article...,เลือดกำเดาไหล ความผิดปกติหลายปัจจัย พร้อมการปฐ...,เลือดกำเดา (Epistaxis) คือ โลหิตที่ไหลออกมาจาก...,health-articles,petcharavej hospital,0.640361
825,https://www.bangpakok8.com/care_blog/view/64,8 สัญญาณเตือนระวังเป็น “ไข้เลือดออก”,8 สัญญาณเตือนระวังเป็น “ไข้เลือดออก” 1.มีไข้สู...,health-articles,bangpakok hospital,0.614980
2166,https://www.bangkokhospital.com/content/surger...,รักษาเลือดกำเดาไหลด้วยการผ่าตัด,ข้อมูลสุขภาพ โรคและการรักษา รักษาเลือดกำเดาไหล...,health-articles,bangkok hospital,0.609974
3001,https://www.samitivejhospitals.com/th/article/...,เหงื่อออกมือ ปัญหากวนใจ,เหงื่อออกมือ ปัญหากวนใจ นพ. ต้นกล้า สุฉันทบุตร...,health-articles,samitivejhospitals,0.579041
855,https://www.bangpakok8.com/care_blog/view/30,“หัวใจขาดเลือดเฉียบพลัน” ภัยเงียบที่น่ากลัว,หัวใจขาดเลือดเฉียบพลัน ภัยเงียบที่น่ากลัว ทำให...,health-articles,bangpakok hospital,0.578738
867,https://www.bangpakok8.com/care_blog/view/7,โรคหลอดเลือดสมอง,NaN,health-articles,bangpakok hospital,0.564693
688,https://www.paolohospital.com/th-TH/center/Art...,ภาวะแผลติดเชื้อ สัญญาณร้ายที่ไม่ควรละเลย!,ภาวะแผลติดเชื้อ สัญญาณร้ายที่ไม่ควรละเลย! โรงพ...,health-articles,paolo hospital,0.554971
3457,https://www.samitivejhospitals.com/th/article/...,ภาวะเลือดออกในสมอง ที่เกิดจากอุบัติเหตุในเด็ก,ภาวะเลือดออกในสมอง ที่เกิดจากอุบัติเหตุในเด็ก ...,health-articles,samitivejhospitals,0.551479
1315,https://www.petcharavejhospital.com/th/Article...,ใส่หน้ากากนานทำให้เลือดเป็นกรดจริงหรือ,หลังจากมีข่าวออกมาว่าทั่วโลกโซเชียลว่า ใส่หน้า...,health-articles,petcharavej hospital,0.546653
